I believe have copied and pasted all relevant code into this notebook - that being said, just in case, I have included all the scripts and things I worked on in the /other/ folder in the .zip file

# Question 1.1

### Prep

In [ ]:
!mkdir data
!cd data
!wget https://community.dur.ac.uk/hubert.shum/comp42315resit.zip
!unzip comp42315resit.zip
!mv ./Data/* .

### Videos to images

In [ ]:
import cv2
import os

path = "./data/"  
path_images = f"{path}./images/"

def convert_video_to_frames(path_input, path_output, name):
    os.makedirs(path_output, exist_ok=True)
    cap = cv2.VideoCapture(path_input)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    count = 0
    while cap.isOpened():
        _, frame = cap.read()
        cv2.imwrite(f"{path_output}./{name}_{count+1}.jpg", frame)
        count = count + 1
        if (count > (video_length-1)):
            cap.release()
            break

dirs = [("game", "trainA"), ("movie", "trainB")]
for d in dirs:
    path_curr = f"{path}./{d[0]}/"
    files = os.listdir(path_curr)
    for f in files:
        path_input = f"{path_curr}./{f}"
        path_output = f"{path_images}./{d[1]}/"
        print(f"Current: {path_input}")
        convert_video_to_frames(path_input, path_output, f[:-4])

### Split data

In [ ]:
import os
import re

path_abs = os.getcwd()
path_images = "./data/images/"
SPLIT = 0.3

movies = ["TheGodfather", "TheSopranos", "TheIrishman"]
files = os.listdir(path_images + "./trainB/")
for movie in movies:
    r = re.compile(f"{movie}.")
    files2 = list(filter(r.match, files))
    num = int(len(files2) * SPLIT)
    files2 = sorted(files2, key=lambda s: int(re.findall(r"\d+", s)[-1]))
    files2 = files2[-num:]
    for f in files2:
        p = f"{path_abs}/data/images/testB/"
        os.makedirs(p, exist_ok=True)
        os.rename(f"{path_abs}/data/images/trainB/{f}", p + f)

files = os.listdir(path_images + "./trainA/")
num = int(len(files) * SPLIT)
files = sorted(files, key=lambda s: int(re.findall(r"\d+", s)[-1]))
files = files[-num:]
for f in files:
    p = f"{path_abs}/data/images/testA/"
    os.makedirs(p, exist_ok=True)
    os.rename(f"{path_abs}/data/images/trainA/{f}", p + f)


In [ ]:
import torchvision.transforms as T
import os
from PIL import Image

path = "./data/images/"
path_new = "./data/images/"
dirs = ["trainA", "trainB"]
for dir in dirs:
    files = os.listdir(path + dir + "/")
    path_temp = path + dir + "/"
    os.makedirs(path_temp, exist_ok=True)
    for f in files:
        img = Image.open(path + dir + "/" + f)
        new_img = T.CenterCrop((512,512))(img)
        new_img.save(path_temp + f)


# Question 2.1

### Prep

In [ ]:
!git clone https://github.com/taesungp/contrastive-unpaired-translation.git

After cloning the git repo, please move the patchL2.py, patchTripletMargin.py, and the modified cut_model.py files to the /models/ folder

In [ ]:
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name g2m --CUT_mode CUT --direction AtoB --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 256
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name m2g --CUT_mode CUT --direction BtoA --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 256
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name m2g_128 --CUT_mode CUT --direction BtoA --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 128
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name g2m_128 --CUT_mode CUT --direction AtoB --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 128
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name m2g_64 --CUT_mode CUT --direction BtoA --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 64
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name g2m_64 --CUT_mode CUT --direction AtoB --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 64
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name m2g_L2 --CUT_mode CUT --direction BtoA --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 256 --patch_L2 True
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name g2m_L2 --CUT_mode CUT --direction AtoB --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 256 --patch_L2 True
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name m2g_Triplet --CUT_mode CUT --direction BtoA --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 256 --patch_TripletMargin True
!python contrastive-unpaired-translation/train.py --dataroot ./data/images/ \
    --name m2g --CUT_mode CUT --direction BtoA --load_size 512 \
    --crop_size 512 --preprocess none \
    --n_epochs 50 --n_epochs_decay 50 --num_patches 256 --patch_TripletMargin True

As a footnote, I would like to mention that I considered using the bounding boxes from 3.1 to change the patch sampling strategy by oversampling regions with faces in them. However, due to limited time, I couldn't implement this.

# Question 3.1

In [ ]:
import cv2
import numpy as np

# Source:
# https://pyimagesearch.com/2020/06/15/opencv-fast-fourier-transform-fft-for-blur-detection-in-images-and-video-streams/
def detect_blur_fft(image, size=60, thresh=0):
    (h, w) = image.shape
    (cX, cY) = (int(w / 2.0), int(h / 2.0))
    fft = np.fft.fft2(image)
    fftShift = np.fft.fftshift(fft)
    fftShift[cY - size:cY + size, cX - size:cX + size] = 0
    fftShift = np.fft.ifftshift(fftShift)
    recon = np.fft.ifft2(fftShift)
    magnitude = 20 * np.log(np.abs(recon))
    mean = np.mean(magnitude)
    return (mean, mean <= thresh)

### Detect faces

In [ ]:
import os
import face_recognition
import numpy as np
from PIL import Image, ImageEnhance

path_images = "./data/images_original/"
path_faces = "./data/faces/"
path_faces_hq = "./data/faces_high_quality/"
dirs = ["trainA", "trainB"]
batch_size = 16

for d in dirs:
    path_input = path_images + d + "/"
    path_output = path_faces + d + "/"
    path_output_hq = path_faces_hq + d + "/"
    os.makedirs(path_output, exist_ok=True)
    os.makedirs(path_output_hq, exist_ok=True)
    files = os.listdir(path_input)
    images = []
    for index_files, f in enumerate(files):
        image = Image.open(path_input + f)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(1.5)
        image = np.array(image)
        images.append(image)
        if len(images) == batch_size:
            face_locations_batch = face_recognition.batch_face_locations(images, batch_size=batch_size)
            for index_batch, face_locations in enumerate(face_locations_batch):
                index = index_files + 1 - batch_size + index_batch
                for index_face, face_location in enumerate(face_locations):
                    top, right, bottom, left = face_location
                    image_face_np = images[index_batch][top:bottom, left:right]
                    image_face = Image.fromarray(image_face_np)
                    enhancer = ImageEnhance.Brightness(image_face)
                    image_face = enhancer.enhance(0.75)
                    image_face = image_face.resize((128,128))
                    image_face.save(f"{path_output}{files[index][:-4]}_{index_face}.jpg")
                    _, blurry = detect_blur_fft(cv2.cvtColor(image_face_np, cv2.COLOR_RGB2GRAY), size=128, thresh=-10)
                    if abs(top - bottom) * abs(left - right) > 4096 and not blurry:
                        image_face.save(f"{path_output_hq}{files[index][:-4]}_{index_face}.jpg")
            images = []

### Split faces dataset

In [ ]:
import face_recognition
import os
import numpy as np
from PIL import Image
import random
import queue

path_train = "./data/images_faces/"
path_test = "./data/images_faces_test/"
dirs = ["trainA", "trainB"]

for d in dirs:
    path_train_temp = path_train + d + "/"
    files = os.listdir(path_train_temp)
    encodings = []
    for f in files:
        image = Image.open(path_train_temp + f)
        try:
            encoding = face_recognition.face_encodings(np.array(image), known_face_locations=[[0,128,128,0]])[0]
            encodings.append(encoding)
        except IndexError:
            print("Face not detected.")

    path_temp_test = path_test + d + "/"
    test_set_indices = []
    test_set_size = int(len(files) * 0.3)

    def find_close_faces(index, visited_face_indices, num):
        new_visited_face_indices = visited_face_indices
        new_visited_face_indices += [index]
        q = queue.Queue()
        q.put(index)
        while not q.empty() and len(new_visited_face_indices) < num:
            index = q.get()
            face_distances = face_recognition.face_distance(encodings[:index] + encodings[index+1:], encodings[index])
            list_indices = np.argsort(face_distances)
            i = 0
            while face_distances[list_indices[i]] <= 0.6 and len(new_visited_face_indices) < num:
                if list_indices[i] >= index and (list_indices[i] + 1) not in new_visited_face_indices:
                    q.put(list_indices[i]+1)
                    new_visited_face_indices += [list_indices[i] + 1]
                elif list_indices[i] < index and list_indices[i] not in new_visited_face_indices:
                    q.put(list_indices[i])
                    new_visited_face_indices += [list_indices[i]]
                i += 1
        return new_visited_face_indices

    while len(test_set_indices) < test_set_size:
        index = random.randint(0, len(encodings))
        while index in test_set_indices:
            index = random.randint(0, len(encodings))
        test_set_indices = find_close_faces(index, test_set_indices, test_set_size)
    
    for i in test_set_indices:
        os.makedirs(path_temp_test, exist_ok=True)
        os.rename(path_train_temp + files[i], path_temp_test + files[i])

# Question 3.2

In [ ]:
!python contrastive-unpaired-translation/train.py --dataroot ./data/images_faces/ \
    --name games2movies_faces --CUT_mode CUT --direction AtoB --load_size 128 \
    --crop_size 128 --preprocess none
!python contrastive-unpaired-translation/train.py --dataroot ./data/images_faces/ \
    --name m2g_faces --CUT_mode CUT --direction BtoA --load_size 128 \
    --crop_size 128 --preprocess none
!python contrastive-unpaired-translation/train.py --dataroot ./data/faces_high_quality/ \
    --name games2movies_faces_hq --CUT_mode CUT --direction AtoB --load_size 128 \
    --crop_size 128 --preprocess none
!python contrastive-unpaired-translation/train.py --dataroot ./data/faces_high_quality/ \
    --name m2g_faces_hq --CUT_mode CUT --direction BtoA --load_size 128 \
    --crop_size 128 --preprocess none

# Question 3.3

I tried a bunch of things for this question, none of them worked, except Albumentations. I haven't included my attempts with AttGAN and PA-GAN. the apply_make_up thing I tried below didn't work well, so i left it out

In [ ]:
import cv2
import albumentations as A
import os
import random
from PIL import Image
import numpy as np
from PIL import Image, ImageDraw
import face_recognition
import shutil

# Source:
# https://github.com/ageitgey/face_recognition/blob/master/examples/digital_makeup.py
def apply_make_up(image):
    face_landmarks_list = face_recognition.face_landmarks(image)
    pil_image = Image.fromarray(image)
    for face_landmarks in face_landmarks_list:
        d = ImageDraw.Draw(pil_image, 'RGBA')

        # Make the eyebrows into a nightmare
        d.polygon(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 128))
        d.polygon(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 128))
        d.line(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 150), width=5)
        d.line(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 150), width=5)

        # Gloss the lips
        d.polygon(face_landmarks['top_lip'], fill=(150, 0, 0, 128))
        d.polygon(face_landmarks['bottom_lip'], fill=(150, 0, 0, 128))
        d.line(face_landmarks['top_lip'], fill=(150, 0, 0, 64), width=8)
        d.line(face_landmarks['bottom_lip'], fill=(150, 0, 0, 64), width=8)

        # Sparkle the eyes
        d.polygon(face_landmarks['left_eye'], fill=(255, 255, 255, 30))
        d.polygon(face_landmarks['right_eye'], fill=(255, 255, 255, 30))

        # Apply some eyeliner
        d.line(face_landmarks['left_eye'] + [face_landmarks['left_eye'][0]], fill=(0, 0, 0, 110), width=6)
        d.line(face_landmarks['right_eye'] + [face_landmarks['right_eye'][0]], fill=(0, 0, 0, 110), width=6)

    return np.array(pil_image)

class MakeUp(A.ImageOnlyTransform):
    def apply(self, img, **params):
        return apply_make_up(img)

# We tested all of Albumentations geometric and photometric transformations
# We include only the transformations that result in realistic images
transform = A.Compose([
    A.OneOf([
        A.AdvancedBlur(),
        A.GaussianBlur(),
        A.MedianBlur(),
        A.MotionBlur(),
    ]),
    A.OneOf([
        A.GaussNoise(),
        A.MultiplicativeNoise(),
        A.ISONoise(),
    ]),
    A.OneOf([
        A.RandomBrightnessContrast(),
        A.RandomToneCurve(),
    ]),

    A.Emboss(),
    A.RingingOvershoot(),
    A.Sharpen(),
    A.OneOf([
        A.CLAHE(),
        A.Equalize(),
    ]),
    A.FancyPCA(),
    A.HorizontalFlip(),
    A.OneOf([
        A.OpticalDistortion(),
        A.GridDistortion(),
        A.ShiftScaleRotate(),
    ]),
])

def create_augmented_dataset(path_in, path_out, augment_chance=0.3):
    files = os.listdir(path_in)
    os.makedirs(path_out, exist_ok=True)
    for f in files:
        shutil.copyfile(path_in + f, path_out + f)
        image = cv2.imread(path_in + f)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        augment = random.random() < augment_chance
        if augment:
            new_image = transform(image=image)["image"]
            Image.fromarray(new_image).save(path_out + f[:-4] + "_a.jpg")
            exit()

create_augmented_dataset("./data/faces_high_quality_bigger/trainA/", "./data/faces_augmented/trainA/")
create_augmented_dataset("./data/faces_high_quality_bigger/trainB/", "./data/faces_augmented/trainB/")
